<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Fran Zautzik
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** `https://github.com/bacchus00/laboratorio-mds`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [2]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 63.2 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [3]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/Bushe/Laboratorio/Lab8/'
except: 
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


In [4]:
df_players = pd.read_csv(path+'stats_players.csv')
df_players.sample(5)

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
7377,Romeu,Portugal,NaN,Res,185,72,Left,31,Medium / Medium,3,...,52,80,64,71,52,47,53,64,41,30
12527,Aldo Arellano,Mexico,NaN,Sub,174,64,Right,21,Low / High,3,...,68,61,55,56,39,39,48,48,51,34
5977,Mateusz Mak,Poland,NaN,Sub,176,64,Right,25,High / Medium,4,...,93,84,53,56,65,66,55,56,51,59
12708,Victor Osimhen,Nigeria,NaN,Res,185,76,Right,18,Medium / Medium,4,...,68,75,63,65,66,55,59,39,65,59
4435,Kazenga LuaLua,DR Congo,NaN,Sub,180,77,Right,26,High / Low,4,...,93,57,60,83,65,76,63,68,64,67


In [5]:
df_salarios = pd.read_csv(path+'salarios.csv')
df_salarios = df_salarios[['Player', 'Weekly Salary']]
df_salarios.sample(5)

,Player,Weekly Salary
2518,Benjamin Goller,598.0
307,Mattia Perin,92628.0
812,Santi Mina,43345.0
9,Toni Kroos,434977.0
1719,Enis Bardhi,14093.0


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [6]:
df_players['National_Position'].value_counts(dropna=False)

NaN    16513
Sub      556
LCB       48
GK        47
RCB       46
LB        39
RB        38
RM        34
LM        32
ST        30
LCM       25
RCM       25
LDM       19
CAM       19
RDM       18
RS        18
LS        18
CB         9
CDM        9
CM         9
RW         7
LW         7
LWB        4
RAM        4
LAM        4
RWB        4
RF         3
LF         3
Name: National_Position, dtype: int64

In [64]:
df_clf_sel = df_players.copy()
df_clf_sel['labels'] = df_clf_sel['National_Position'].apply(lambda x: 1 if x==x else 0)
df_clf_sel.drop(columns=['National_Position'], inplace=True)
df_clf_sel['labels'].value_counts(dropna=False)

0    16513
1     1075
Name: labels, dtype: int64

In [66]:
df_clf_sel.shape

(17588, 39)

Para los seleccionados tenemos 1075 y para no seleccionados 16513, es decir hay un gran desbalance de aproximadamente 1:16 de seleccionados a no seleccionados.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [X] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [X] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [X] Crear 3 ``pipelines`` con diferentes clasificadores.
- [X] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [X] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [8]:
cat_nom = [
    'Name', 
    'Nationality', 
    'Preffered_Foot',
    'Club_Position', 
    'Work_Rate', 
    'Weak_foot',
    'Skill_Moves'
] #OneHotEncoder

num_norm = [
    'Height', 
    'Weight',
    'Age',
    'Aggression',
    'Reactions',
    'Vision',
    'Composure',
    'Acceleration',
    'Speed',
    'Strength',
    'Balance',
    'Agility',
    'Jumping'
] #StandardScaler

num_unk = [
    'Ball_Control', 
    'Dribbling', 
    'Marking', 
    'Sliding_Tackle', 
    'Standing_Tackle', 
    'Interceptions', 
    'Crossing',
    'Short_Pass',
    'Long_Pass',
    'Stamina',
    'Heading',
    'Shot_Power',
    'Finishing',
    'Long_Shots',
    'Curve',
    'Freekick_Accuracy',
    'Penalties',
    'Volleys'
] #MinMax

t_cat_nom = ('t_cat_nom_ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), cat_nom)
t_num_norm = ('t_num_norm_ss', StandardScaler(), num_norm)
t_num_unk = ('t_num_unk_mm', MinMaxScaler(), num_unk)
col_trans = ColumnTransformer([t_cat_nom, t_num_norm, t_num_unk], remainder='passthrough')
col_trans.set_output(transform='pandas')

ColumnTransformer(remainder='passthrough',
                  transformers=[('t_cat_nom_ohe',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['Name', 'Nationality', 'Preffered_Foot',
                                  'Club_Position', 'Work_Rate', 'Weak_foot',
                                  'Skill_Moves']),
                                ('t_num_norm_ss', StandardScaler(),
                                 ['Height', 'Weight', 'Age', 'Aggression',
                                  'Reactions', 'Vision', 'Composure',
                                  'Acceleration', 'Speed', 'Strength',
                                  'Balance', 'Agility', 'Jumping']),
                                ('t_num_unk_mm', MinMaxScaler(),
                                 ['Ball_Control', 'Dribbling', 'Marking',
                                  'Sliding_Tackle', 'Standing_Tackle',
                                  'Interceptions', 'Crossing', 'Short_Pass',
                                  'Long_Pass', 'Stamina', 'Heading',
                                  'Shot_Power', 'Finishing', 'Long_Shots',
                                  'Curve', 'Freekick_Accuracy', 'Penalties',
                                  'Volleys'])])

Las variables categoricas, tras no encontrar ningun orden intuitivo de sus niveles, fuero asumidas como simplemente variables nominales ('cat_nom') por lo que para ellas se opto por usar OneHotEncoder para su transformacion.

En cuanto a las variables numericas primero se puedo identificar que habian algunas con una clara semejanza a la distribucion normal ('num_norm') por lo que se opto por la el StandardScaler para su transformacion.

Para las otras variables numericas, dado que o no tenian una distribucion claramente identificable o tenian una forma bimodal, se utilizo el MinMax para la transformacion.

In [10]:
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier

svm_lineal = LinearSVC()
knc = KNeighborsClassifier()
svm_poly = SVC(kernel='poly')

pipeline1 = Pipeline(
    steps=[
        ('transformaciones', col_trans),
        ('svm_lineal', svm_lineal)
    ]
)

pipeline2 = Pipeline(
    steps=[
        ('transformaciones', col_trans),
        ('knc', knc)
    ]
)

pipeline3 = Pipeline(
    steps=[
        ('transformaciones', col_trans),
        ('svm_poly', svm_poly)
    ]
)

A continuacion, a la hora de definir los pipelines, se escogio 3 modelos de classificacion (pues es data etiquetada) segun la guia de sklearn. Considerando que son menos de 100k datos se opto por usar 'LinearSVC' (svm lineal para clasificaion). Luego, asumiendo que este no tendria un buen desepmeño y que no estamos trabajando con texto se paso a 'KNeighborsClassifier'. Finalmente asumiendo nuevamente que el modelo anterior no funcionaria se utilizo 'SVC' con un kernel polinomial (es decir no lineal).

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [X] Separar el conjunto de datos en entrenamiento y de prueba.
- [X] Entrenar los `pipelines`.
- [X] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [11]:
X = df_clf_sel.loc[:, ~df_clf_sel.columns.isin(['labels'])]
y = df_clf_sel['labels']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [13]:
pipeline1.fit(X_train, y_train)
pipeline2.fit(X_train, y_train)
pipeline3.fit(X_train, y_train)

Pipeline(steps=[('transformaciones',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('t_cat_nom_ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Name', 'Nationality',
                                                   'Preffered_Foot',
                                                   'Club_Position', 'Work_Rate',
                                                   'Weak_foot',
                                                   'Skill_Moves']),
                                                 ('t_num_norm_ss',
                                                  StandardScaler(),
                                                  ['Height', 'Weight', 'Age',
                                                   'Aggression', 'Reactions',
                                                   'Vi...
                                                   'Acceleration', 'Speed',
                                                   'Strength', 'Balance',
                                                   'Agility', 'Jumping']),
                                                 ('t_num_unk_mm',
                                                  MinMaxScaler(),
                                                  ['Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Interceptions', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Stamina', 'Heading',
                                                   'Shot_Power', 'Finishing',
                                                   'Long_Shots', 'Curve',
                                                   'Freekick_Accuracy',
                                                   'Penalties', 'Volleys'])])),
                ('svm_poly', SVC(kernel='poly'))])

In [14]:
y_pred = pipeline1.predict(X_test)
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred), "\n")
print(classification_report(y_test, y_pred))

Matriz de confusión

 [[5392   64]
 [ 223  126]] 

              precision    recall  f1-score   support

           0       0.96      0.99      0.97      5456
           1       0.66      0.36      0.47       349

    accuracy                           0.95      5805
   macro avg       0.81      0.67      0.72      5805
weighted avg       0.94      0.95      0.94      5805



In [15]:
y_pred = pipeline2.predict(X_test)
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred), "\n")
print(classification_report(y_test, y_pred))

Matriz de confusión

 [[5429   27]
 [ 334   15]] 

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5456
           1       0.36      0.04      0.08       349

    accuracy                           0.94      5805
   macro avg       0.65      0.52      0.52      5805
weighted avg       0.91      0.94      0.91      5805



In [16]:
y_pred = pipeline3.predict(X_test)
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred), "\n")
print(classification_report(y_test, y_pred))

Matriz de confusión

 [[5441   15]
 [ 322   27]] 

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5456
           1       0.64      0.08      0.14       349

    accuracy                           0.94      5805
   macro avg       0.79      0.54      0.55      5805
weighted avg       0.93      0.94      0.92      5805



**Respuesta**:

Del resultado del primer modelo usando LinearSVC es posible ver que el modelo no tiene un buen acurracy (0.95) pero si consideramos los resultados para la clase 1 es posible notar que el modelo no es muy bueno reconociendo jugadores de esta clase.

Para los otrso dos modelos es posible notar que el accuracy disminuyo (0.94) y el desempeño para el reconocimiento de seleccionados dismiuyo considerablemente por lo que la mejor alternativa para este problema parece ser el primer modelo propuesto.

El mal desempeño a la hora reconocer ejemplos de seleccionados puede deberse al gran desbalance que existe entre las clases mencionado anteriormente frente a lo cula podria adoptarse una tecnica de subsamplng o oversampling. Otra razon para el mal desempeño puede deberse a que quiza no todas las variables utilizadas para entrenar sean utilies para predecir las clases o que hayan muchas variables correlacionadas generando ruido. Frente lo anterior seria util probar los modelos usando distintos subconjuntos de los datos.

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [X] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [X] Cuente cuantos por clase quedan.
- [X] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [X] Comente los resultados obtenidos.

**Respuesta:**

In [17]:
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']

In [18]:
df_clf_pos = df_players[df_players['Club_Position'].isin(ataque+central_ataque+central+central_defensa+defensa+arquero)].copy()
df_clf_pos['National_Position'] = df_clf_pos['National_Position'].apply(lambda x: x if x==x else 'None')
df_clf_pos['Club_Position'].replace(to_replace=ataque, value='ataque', inplace=True)
df_clf_pos['Club_Position'].replace(to_replace=ataque, value='ataque', inplace=True)
df_clf_pos['Club_Position'].replace(to_replace=central_ataque, value='central_ataque', inplace=True)
df_clf_pos['Club_Position'].replace(to_replace=central, value='central', inplace=True)
df_clf_pos['Club_Position'].replace(to_replace=central_defensa, value='central_defensa', inplace=True)
df_clf_pos['Club_Position'].replace(to_replace=defensa, value='defensa', inplace=True)
df_clf_pos['Club_Position'].replace(to_replace=arquero, value='arquero', inplace=True)
df_clf_pos['Club_Position'].value_counts()

defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: Club_Position, dtype: int64

In [63]:
df_clf_pos.shape

(3939, 39)

In [19]:
cat_nom = [
    'Name', 
    'Nationality', 
    'Preffered_Foot',
    'National_Position', 
    'Work_Rate', 
    'Weak_foot',
    'Skill_Moves'
] #OneHotEncoder

num_norm = [
    'Height', 
    'Weight', 
    'Age',
    'Aggression',
    'Reactions',
    'Vision',
    'Composure',
    'Acceleration',
    'Speed',
    'Strength',
    'Balance',
    'Agility',
    'Jumping'
] #StandardScaler

num_unk = [
    'Ball_Control', 
    'Dribbling', 
    'Marking', 
    'Sliding_Tackle', 
    'Standing_Tackle', 
    'Interceptions', 
    'Crossing',
    'Short_Pass',
    'Long_Pass',
    'Stamina',
    'Heading',
    'Shot_Power',
    'Finishing',
    'Long_Shots',
    'Curve',
    'Freekick_Accuracy',
    'Penalties',
    'Volleys'
] #MinMax

t_cat_nom = ('t_cat_nom_ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), cat_nom)
t_num_norm = ('t_num_norm_ss', StandardScaler(), num_norm)
t_num_unk = ('t_num_unk_mm', MinMaxScaler(), num_unk)
col_trans = ColumnTransformer([t_cat_nom, t_num_norm, t_num_unk], remainder='passthrough')
col_trans.set_output(transform='pandas')

ColumnTransformer(remainder='passthrough',
                  transformers=[('t_cat_nom_ohe',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['Name', 'Nationality', 'Preffered_Foot',
                                  'National_Position', 'Work_Rate', 'Weak_foot',
                                  'Skill_Moves']),
                                ('t_num_norm_ss', StandardScaler(),
                                 ['Height', 'Weight', 'Age', 'Aggression',
                                  'Reactions', 'Vision', 'Composure',
                                  'Acceleration', 'Speed', 'Strength',
                                  'Balance', 'Agility', 'Jumping']),
                                ('t_num_unk_mm', MinMaxScaler(),
                                 ['Ball_Control', 'Dribbling', 'Marking',
                                  'Sliding_Tackle', 'Standing_Tackle',
                                  'Interceptions', 'Crossing', 'Short_Pass',
                                  'Long_Pass', 'Stamina', 'Heading',
                                  'Shot_Power', 'Finishing', 'Long_Shots',
                                  'Curve', 'Freekick_Accuracy', 'Penalties',
                                  'Volleys'])])

In [20]:
pipeline_pos = Pipeline(
    steps=[
        ('transformaciones', col_trans),
        ('svm_lineal', svm_lineal)
    ]
)

Nuevamente considerando la guia se sklearn, tratandose de un problema de clasificacion y con menos de 100k datos se opto por utilizar LinearSVC.

In [21]:
X = df_clf_pos.loc[:, ~df_clf_pos.columns.isin(['Club_Position'])]
y = df_clf_pos['Club_Position']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [22]:
pipeline_pos.fit(X_train, y_train)

Pipeline(steps=[('transformaciones',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('t_cat_nom_ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Name', 'Nationality',
                                                   'Preffered_Foot',
                                                   'National_Position',
                                                   'Work_Rate', 'Weak_foot',
                                                   'Skill_Moves']),
                                                 ('t_num_norm_ss',
                                                  StandardScaler(),
                                                  ['Height', 'Weight', 'Age',
                                                   'Aggression', 'Reactions'...
                                                   'Acceleration', 'Speed',
                                                   'Strength', 'Balance',
                                                   'Agility', 'Jumping']),
                                                 ('t_num_unk_mm',
                                                  MinMaxScaler(),
                                                  ['Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Interceptions', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Stamina', 'Heading',
                                                   'Shot_Power', 'Finishing',
                                                   'Long_Shots', 'Curve',
                                                   'Freekick_Accuracy',
                                                   'Penalties', 'Volleys'])])),
                ('svm_lineal', LinearSVC())])

In [23]:
y_pred = pipeline_pos.predict(X_test)
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred), "\n")
print(classification_report(y_test, y_pred))

Matriz de confusión

 [[207   0   0   0   0   0]
 [  0 123  16  10   0   0]
 [  0  19 184  40   6  44]
 [  0  17 111  55   4   7]
 [  0   0   9   4   9  56]
 [  0   0  12   0   5 362]] 

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       207
         ataque       0.77      0.83      0.80       149
        central       0.55      0.63      0.59       293
 central_ataque       0.50      0.28      0.36       194
central_defensa       0.38      0.12      0.18        78
        defensa       0.77      0.96      0.85       379

       accuracy                           0.72      1300
      macro avg       0.66      0.63      0.63      1300
   weighted avg       0.70      0.72      0.70      1300



**Respuesta**:

El modelo obtuvo un accuracy de 0.72 lo cual es medianamente bueno. Observando los resultados para las distintas clases notamos que los aqueros son facilmente reconocidos, los ataques y defensas los identifica medianamente bien, pero con las clases central tiene mucha mas dificultad clasificando correctamente, particularmente para central_defensa que coincidentemente es la clase con menos ejemplos. El pequeño numero de ejemplo (<4.000) tambien es posible que sea uno de los factores que explican el mal desempeño del modelo.

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

La metrica R2 corresponde al coeficiente de determinacion del modelo y corresponde al proporcio de variacion de la variable dependiente (sueldo en este caso) que es predecible a partir de las variables dependientes (las demas variables).

In [54]:
df_reg = df_players.merge(df_salarios, left_on='Name', right_on='Player').drop(columns=['Player'])
df_reg['National_Position'] = df_reg['National_Position'].apply(lambda x: x if x==x else 'None')
df_reg.sample(5)

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Weekly Salary
120,Javier Pastore,Argentina,None,Sub,187,78,Right,27,High / Medium,4,...,49,63,78,73,76,83,69,75,78,140954.0
480,Christoph Kramer,Germany,None,Sub,191,76,Right,26,High / High,3,...,51,58,53,41,40,54,51,47,49,65247.0
587,Terence Kongolo,Netherlands,None,LB,188,70,Left,23,Medium / High,4,...,85,74,67,27,52,63,44,42,40,46177.0
1410,Anders Lindegaard,Denmark,None,Sub,193,79,Right,32,Medium / Medium,3,...,74,14,28,16,19,11,18,42,13,9895.0
1405,William,Brazil,None,Sub,182,80,Right,25,Medium / Medium,3,...,72,67,69,69,65,70,58,70,64,42976.0


In [55]:
df_reg.shape

(1866, 40)

In [56]:
cat_nom = [
    'Name',
    'Club_Position',
    'National_Position',
    'Nationality', 
    'Preffered_Foot',
    'National_Position', 
    'Work_Rate', 
    'Weak_foot',
    'Skill_Moves'
] #OneHotEncoder

num_norm = [
    'Height', 
    'Weight', 
    'Age',
    'Aggression',
    'Reactions',
    'Vision',
    'Composure',
    'Acceleration',
    'Speed',
    'Strength',
    'Balance',
    'Agility',
    'Jumping'
] #StandardScaler

num_unk = [
    'Ball_Control', 
    'Dribbling', 
    'Marking', 
    'Sliding_Tackle', 
    'Standing_Tackle', 
    'Interceptions', 
    'Crossing',
    'Short_Pass',
    'Long_Pass',
    'Stamina',
    'Heading',
    'Shot_Power',
    'Finishing',
    'Long_Shots',
    'Curve',
    'Freekick_Accuracy',
    'Penalties',
    'Volleys'
] #MinMax

t_cat_nom = ('t_cat_nom_ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), cat_nom)
t_num_norm = ('t_num_norm_ss', StandardScaler(), num_norm)
t_num_unk = ('t_num_unk_mm', MinMaxScaler(), num_unk)
col_trans = ColumnTransformer([t_cat_nom, t_num_norm, t_num_unk], remainder='passthrough')
col_trans.set_output(transform='pandas')

ColumnTransformer(remainder='passthrough',
                  transformers=[('t_cat_nom_ohe',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['Name', 'Club_Position', 'National_Position',
                                  'Nationality', 'Preffered_Foot',
                                  'National_Position', 'Work_Rate', 'Weak_foot',
                                  'Skill_Moves']),
                                ('t_num_norm_ss', StandardScaler(),
                                 ['Height', 'Weight', 'Age', 'Aggression',
                                  'Reactions'...ion', 'Composure',
                                  'Acceleration', 'Speed', 'Strength',
                                  'Balance', 'Agility', 'Jumping']),
                                ('t_num_unk_mm', MinMaxScaler(),
                                 ['Ball_Control', 'Dribbling', 'Marking',
                                  'Sliding_Tackle', 'Standing_Tackle',
                                  'Interceptions', 'Crossing', 'Short_Pass',
                                  'Long_Pass', 'Stamina', 'Heading',
                                  'Shot_Power', 'Finishing', 'Long_Shots',
                                  'Curve', 'Freekick_Accuracy', 'Penalties',
                                  'Volleys'])])

In [57]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge()

pipeline_reg = Pipeline(
    steps=[
        ('transformaciones', col_trans),
        ('ridge_reg', ridge_reg)
    ]
)

Considerando la guia de sklearn tenemos que como es un problema de regression, con menos de 100k filas y hartas variables explicativas una buena opcion es usar RidgeRegression. 

In [58]:
X = df_reg.loc[:, ~df_reg.columns.isin(['Weekly Salary'])]
y = df_reg['Weekly Salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [59]:
pipeline_reg.fit(X_train, y_train)

Pipeline(steps=[('transformaciones',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('t_cat_nom_ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Name', 'Club_Position',
                                                   'National_Position',
                                                   'Nationality',
                                                   'Preffered_Foot',
                                                   'National_Position',
                                                   'Work_Rate', 'Weak_foot',
                                                   'Skill_Moves']),
                                                 ('t_num_norm_ss',
                                                  StandardScaler(),
                                                  ['Height', 'We...
                                                   'Acceleration', 'Speed',
                                                   'Strength', 'Balance',
                                                   'Agility', 'Jumping']),
                                                 ('t_num_unk_mm',
                                                  MinMaxScaler(),
                                                  ['Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Interceptions', 'Crossing',
                                                   'Short_Pass', 'Long_Pass',
                                                   'Stamina', 'Heading',
                                                   'Shot_Power', 'Finishing',
                                                   'Long_Shots', 'Curve',
                                                   'Freekick_Accuracy',
                                                   'Penalties', 'Volleys'])])),
                ('ridge_reg', Ridge())])

In [60]:
pipeline_reg.score(X_test, y_test)

0.365802057966678

**Respuesta**:

Del resultado del modelo podemos darnos cuenta que el coeficiente de determinacion es muy bajos, obteniendose un R2 de solo 0.37, es decir podemos predecir solo una muy pequeña porcion de la variabllidad de los sueldos a partir de las variables escogidas. Lo anterior puede deberse en parte a que se tienen muy pocos ejemplos (<2.000) como para una buena capacidad de generalizacion. Por otro lado es posible que sea mejor utilizar un subconjunto de las variables explicativas puesto que puede que algunas no esten aportando informacion adicional (en el caso de variables correlacionadas) o util para predecir el sueldo. Adicionalmente es posible que simplemente las caracteristicas con las que contamos simplemente no son suficiente informacion como para predecir el sueldo.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>